# Data Exploration & Cleaning

## Importing Modules

In [57]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
from statsmodels.datasets import get_rdataset
import sklearn.model_selection as skm

from sklearn.tree import (DecisionTreeClassifier as DTC,
DecisionTreeRegressor as DTR,
plot_tree,
export_text)
from sklearn.metrics import (accuracy_score ,
log_loss)
from sklearn.ensemble import \
(RandomForestRegressor as RF,
GradientBoostingRegressor as GBR)
import seaborn as sns; sns.set()

## Reading in Data

In [58]:
df = pd.read_csv('data/employee-data.csv')
print(df.columns)
df

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In this case, I want to predict attrition, whether they've left the company or not. Since this is a classification problem, a ML classification approach will be used. 

In [59]:
print([col for col in df.columns if df[col].nunique() == 1]) # Print all columns where there's only one unique value (whole col is just one val)

['EmployeeCount', 'Over18', 'StandardHours']


- As shown, the columns `'Over18', 'EmployeeCount'` and `'StandardHours'` only have one unique value, and so can be dropped from the dataframe as they're not really useful

In [73]:
# What other columns aren't useful for interpretability/the model?

df.iloc[: , 0 : 10] # Display first 10 cols
# EmployeeCount, EmployeeNumber not needed 
df.iloc[: , 10 : 20] # Display 10:20 cols
df.iloc[: , 20 : 30] # Display 20:30 cols
# Over18, standardHours not needed
df.iloc[:, 30:40] # Display 30:40 rows (Last cols 30 - 35)

df = df.drop(columns=['EmployeeCount', 'EmployeeNumber', 'StandardHours', 'Over18'])

In [60]:
df.select_dtypes(include=['object', 'string'])

,Attrition,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18,OverTime
0,Yes,Travel_Rarely,Sales,Life Sciences,Female,Sales Executive,Single,Y,Yes
1,No,Travel_Frequently,Research & Development,Life Sciences,Male,Research Scientist,Married,Y,No
2,Yes,Travel_Rarely,Research & Development,Other,Male,Laboratory Technician,Single,Y,Yes
3,No,Travel_Frequently,Research & Development,Life Sciences,Female,Research Scientist,Married,Y,Yes
4,No,Travel_Rarely,Research & Development,Medical,Male,Laboratory Technician,Married,Y,No
...,...,...,...,...,...,...,...,...,...
1465,No,Travel_Frequently,Research & Development,Medical,Male,Laboratory Technician,Married,Y,No
1466,No,Travel_Rarely,Research & Development,Medical,Male,Healthcare Representative,Married,Y,No
1467,No,Travel_Rarely,Research & Development,Life Sciences,Male,Manufacturing Director,Married,Y,Yes
1468,No,Travel_Frequently,Sales,Medical,Male,Sales Executive,Married,Y,No


In [61]:
for col in df.select_dtypes(include=['object', 'string']).columns:
    print(f"Unique values in '{col}': {df[col].unique()}")

Unique values in 'Attrition': ['Yes' 'No']
Unique values in 'BusinessTravel': ['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']
Unique values in 'Department': ['Sales' 'Research & Development' 'Human Resources']
Unique values in 'EducationField': ['Life Sciences' 'Other' 'Medical' 'Marketing' 'Technical Degree'
 'Human Resources']
Unique values in 'Gender': ['Female' 'Male']
Unique values in 'JobRole': ['Sales Executive' 'Research Scientist' 'Laboratory Technician'
 'Manufacturing Director' 'Healthcare Representative' 'Manager'
 'Sales Representative' 'Research Director' 'Human Resources']
Unique values in 'MaritalStatus': ['Single' 'Married' 'Divorced']
Unique values in 'Over18': ['Y']
Unique values in 'OverTime': ['Yes' 'No']


In [62]:
df

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [ ]:
df._get_numeric_data()

In [25]:
features = df.drop(columns=['Attrition']).columns
features

Index(['Age', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'Gender',
       'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole',
       'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate',
       'NumCompaniesWorked', 'Over18', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,26,991,6,3,3,71,3,1,4,2659,...,3,3,1,3,2,3,3,2,0,2
1,43,185,10,4,3,33,3,1,4,2455,...,3,1,0,9,5,3,8,7,1,7
2,30,438,18,3,1,75,3,1,3,2632,...,3,3,0,5,4,2,5,4,0,4
3,37,1319,6,3,3,51,4,2,1,5974,...,3,1,2,13,2,3,7,7,6,7
4,25,688,3,3,1,91,3,1,1,4031,...,3,3,1,6,5,3,2,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,26,482,1,2,2,90,2,1,3,2933,...,3,3,1,1,3,2,1,0,1,0
1466,41,143,4,3,1,56,3,2,2,9355,...,3,3,0,8,5,3,8,7,7,7
1467,35,464,4,2,3,75,3,1,4,1951,...,3,3,1,1,3,3,1,0,0,0
1468,32,977,2,3,4,45,3,2,2,5470,...,3,3,2,10,4,2,9,5,1,6


In [6]:
df['Attrition'].value_counts()

Attrition
No     1233
Yes     237
Name: count, dtype: int64

As we can see, the response is extremely unbalanced, with 1223 values of no and 237 values of yes. Machine learning literature has three solutions generally presented as a solution to this:

1. Restore balance by undersampling overrepresented class and oversampling underrepresented class.
2. ALTERNATIVELY/MODIFICATION to (1) - [SMOTE](https://arxiv.org/abs/1106.1813) (Synthetic Minority Oversampling Technique): 
3. Increase the cost/penalty for misclassifying 'Yes' answers, to avoid the model from overpicking 'No'
4. Replace the accuracy with 'balanced accuracy', the mean of class-specific accuracies: $\phi := \frac{1}{2}(\pi^+ + \pi^-)$, where $\pi^+$ and $\pi^-$ represent the accuracy on the positive and negative samples, respectively. If the classifier performs equally well on either class, this term reduces to the conventional accuracy (i.e., the number of correct predictions divided by the total number of predictions). In contrast, if the conventional accuracy is above chance only because the classifier takes advantage of an imbalanced test set, then the balanced accuracy, as appropriate, will drop to chance.

For now, we'll ignore the class imbalance and see if it plays a role in the model's fit.

Let's approach this problem using tree-based methods.

## Machine Learning Workflow

In [28]:
# Splitting data

df = df.sample(frac = 1).reset_index(drop = True) # Randomly shuffles rows of df, drops old index col and makes new one
df_train = df.head(1000) # Takes first 1000 rows as training data
df_test = df.tail(470) # Takes last 470 rows as test data

## Fitting Classification Trees

### Basic Decision Tree

In [22]:
X = df[features]
y = df['Attrition']

#print(X)
#print(y)